In [16]:
import tensorflow as tf
from tensorflow.keras import layers,datasets,models
from tensorflow.keras.regularizers import L2
# from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

In [ ]:
!pip install keras-tuner

# Using keras tuner we will find out the optimal number of filter size and units in neural network

In [49]:
def build_model(hp):
    model = models.Sequential()

    # convolutional block 1
    model.add(layers.Conv2D(
        filters=hp.Int('conv1_filters', min_value = 32, max_value=128,step =32),
        kernel_size=(3,3),
        activation='relu',
        padding='same',
        input_shape =(28,28,3)),
    model.add(layers.MaxPooling2D(pool_size=(2,2))))


    # convolutional block 2

    model.add(layers.Conv2D(filters=hp.Int('conv2_filters',min_value =32, max_value=128,step=32),
                  kernel_size=(3,3),
                  activation='relu'))

    model.add(layers.MaxPooling2D(pool_size=(2,2)))


    # Dense layer
    model.add(layers.Dense(units = hp.Int('dense_units' ,
                                          min_value=64, 
                                          max_value=512, step=64),
                           activation='relu'))


    # output layer
    model.add(layers.Flatten())
    model.add( layers.Dense(10,activation='softmax'))




    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])



    return model
    
    

    
 

# load dataset

In [50]:
(x_train,y_train),(x_test,y_test) = datasets.cifar10.load_data()

In [51]:
x_train.shape

(50000, 32, 32, 3)

In [52]:
x_train =x_train.reshape(-1,32,32,3).astype('float')/255.0
x_test =x_test.reshape(-1,32,32,3).astype('float')/255.0       # 1 for grayscale

# Run tuner

In [53]:
tuner =  kt.Hyperband( build_model,
                       objective='val_accuracy',
                       max_epochs=10,
                       directory ='cnn_tunning',
                       project_name='cifar10',
                       )

tuner.search(x_train,y_train,epochs=10,validation_data=(x_test,y_test))
                       
                      

Trial 15 Complete [00h 02m 52s]
val_accuracy: 0.6571999788284302

Best val_accuracy So Far: 0.6636999845504761
Total elapsed time: 00h 18m 38s

Search: Running Trial #16

Value             |Best Value So Far |Hyperparameter
64                |64                |conv1_filters
96                |96                |conv2_filters
512               |384               |dense_units
4                 |4                 |tuner/epochs
0                 |2                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
0                 |1                 |tuner/round

Epoch 1/4
 164/1563 ━━━━━━━━━━━━━━━━━━━━ 38s 28ms/step - accuracy: 0.2329 - loss: 2.0862

KeyboardInterrupt: 

In [57]:
best_model = tuner.get_best_models(1)[0]
best_hyper_parameter = tuner.get_best_hyperparameters(1)[0]


print('best conv1 filters', best_hyper_parameter.get('conv1_filters'))
print('best conv2 filters', best_hyper_parameter.get('conv2_filters'))
print('best dense units',best_hyper_parameter.get('dense_units'))

best conv1 filters 64
best conv2 filters 96
best dense units 384


C:\Users\gauri\OneDrive\Desktop\Deep_Learning\tfkenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\gauri\OneDrive\Desktop\Deep_Learning\tfkenv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
